# Initialization and Dataset Loading

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -q "/content/drive/My Drive/data/tiny-imagenet-200.zip"

replace tiny-imagenet-200/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [3]:
loop_no =10

In [4]:
### Imports ###
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras.engine.topology import Network
from keras.layers import *
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K

import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import os
import random
import scipy.misc
from tqdm import *
import time

%matplotlib inline

In [5]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [6]:
#to see gpu configuration
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed May 12 05:27:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
### Constants ###
DATA_DIR = "/content/tiny-imagenet-200"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

IMG_SHAPE = (64, 64)

In [8]:
def load_dataset_small(num_images_per_class_train=100, num_images_test=1000):
    """Loads training and test datasets, from Tiny ImageNet Visual Recogition Challenge.

    Arguments:
        num_images_per_class_train: number of images per class to load into training dataset.
        num_images_test: total number of images to load into training dataset.
    """
    X_train = []
    X_test = []
    
    # Create training set.
    for c in os.listdir(TRAIN_DIR):
        c_dir = os.path.join(TRAIN_DIR, c, 'images')
        c_imgs = os.listdir(c_dir)
        random.shuffle(c_imgs)
        #for img_name_i in c_imgs[0:num_images_per_class_train]:
        for img_name_i in c_imgs:
            img_i = image.load_img(os.path.join(c_dir, img_name_i))
            x = image.img_to_array(img_i)
            X_train.append(x)
    random.shuffle(X_train)
    
    # Create test set.
    test_dir = os.path.join(TEST_DIR, 'images')
    test_imgs = os.listdir(test_dir)
    random.shuffle(test_imgs)
    #for img_name_i in test_imgs[0:num_images_test]:
    for img_name_i in test_imgs:
        img_i = image.load_img(os.path.join(test_dir, img_name_i))
        x = image.img_to_array(img_i)
        X_test.append(x)

    # Return train and test data as numpy arrays.
    return np.array(X_train), np.array(X_test)

In [9]:
# Load dataset.
X_train_orig, X_test_orig = load_dataset_small()

# Normalize image vectors.
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Print statistics.
print ("Number of training examples = " + str(X_train.shape[0]))
print ("Number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape)) # Should be (train_size, 64, 64, 3).

Number of training examples = 100000
Number of test examples = 10000
X_train shape: (100000, 64, 64, 3)


In [10]:
#training set lagbe na bro
del X_train
del X_train_orig, X_test_orig

In [11]:
# We split testing set into two halfs.
# S: secret image
input_S = X_test[0:X_test.shape[0] // 2]

# C: cover image
input_C = X_test[X_test.shape[0] // 2:]

In [12]:
def lr_schedule(epoch_idx):
    if epoch_idx < 200:
        return 0.001
    elif epoch_idx < 400:
        return 0.0003
    elif epoch_idx < 600:
        return 0.0001
    else:
        return 0.00003

# Baluja

### Model Implementation

In [13]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = K.sum(K.square(s_true - s_pred))
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))

    x = input_S
    
    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = concatenate([input_C, x])
    
    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)
    
    return Model(inputs=[input_S, input_C], #changed
                 outputs=output_Cprime,
                 name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    
    # Reveal network
    reveal_input = Input(shape=(input_size))

    # Adding Gaussian noise with 0.01 standard deviation.
    x = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed
  
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Sprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)
    
    if not fixed:
        return Model(inputs=reveal_input,
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs= reveal_input,   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C]) #changed
    output_Sprime = decoder(output_Cprime) #changed

    autoencoder = Model(inputs=[input_S, input_C],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [14]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [15]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/baluja/baluja_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/baluja/baluja_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/baluja/baluja_reveal_model_100k_50e.hdf5')

In [16]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [17]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [18]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [19]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  13.121557148000011
Encryption Time :  5.912254163099897
Decryption Time :  4.0842034951000645
pixel_s_error :  8.343199729919434
pixel_c_error:  8.590557098388672
ssim_s_error :  0.9396273493766785
ssim_c_error :  0.9048897624015808
psnr_s_error :  30.237600326538086
psnr_c_error :  29.723115921020508


# Key Concatenation

### Model Implementation







In [20]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = rev_loss(s_true, s_pred)
    c_loss = rev_loss(c_true, c_pred)
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))
    input_K = Input(shape=(input_size))

    # input_K = Input(shape=(81,)) #changed
    # filter = Reshape((3, 3, 3, 3))(input_K)
    # x = tf.nn.conv2d(input_S, filter, strides = (1, 1, 1, 1), padding='VALID')


    #x = tf.math.multiply(input_S, input_K)
    x = concatenate([input_S, input_K])
    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = concatenate([input_C, x])
    
    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)
    
    return Model(inputs=[input_S, input_C, input_K], #changed
                 outputs=output_Cprime,
                 name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    
    # Reveal network
    reveal_input = Input(shape=(input_size))
    input_K = Input(shape=(input_size))


    # Adding Gaussian noise with 0.01 standard deviation.
    input_with_noise = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed

    x = concatenate([input_with_noise, input_K])
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Sprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)
    
    if not fixed:
        return Model(inputs=[reveal_input,input_K],
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs=[reveal_input,input_K],   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    input_K = Input(shape=(input_size))  #changed
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C, input_K]) #changed
    output_Sprime = decoder([output_Cprime, input_K]) #changed

    autoencoder = Model(inputs=[input_S, input_C, input_K],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [21]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [22]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/key_concatenation/key_concatenation_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/key_concatenation/key_concatenation_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/key_concatenation/key_concatenation_reveal_model_100k_50e.hdf5')

In [23]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [24]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [25]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [26]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  input_K = np.random.randint(256, size=(len(input_S), 64, 64, 3))
  input_K = input_K/255.

  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime, input_K])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  10.388874442900123
Encryption Time :  6.22470604150003
Decryption Time :  4.3036049101000575
pixel_s_error :  5.727689123153686
pixel_c_error:  7.282300186157227
ssim_s_error :  0.9658628702163696
ssim_c_error :  0.9189670205116272
psnr_s_error :  33.38980331420898
psnr_c_error :  31.170940017700197


# Pixel Shuffling

### Model Implementation

In [27]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = K.sum(K.square(s_true - s_pred))
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))
    input_K = Input(shape=(12288,))

    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(tf.shape(integer_K)[0]), axis = 1)
    i1 = tf.tile(i1, [1, 12288])
    indices = tf.stack([i1, integer_K], axis=-1)
    tensor = Flatten()(input_S)
    shape = tf.shape(tensor)
    scattered_tensor = tf.scatter_nd(indices, tensor, shape)
    x = Reshape((64,64,3))(scattered_tensor)
    

    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = concatenate([input_C, x])
    
    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)
    
    return Model(inputs=[input_S, input_C, input_K], #changed
                 outputs=output_Cprime,
                 name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    
    # Reveal network
    reveal_input = Input(shape=(input_size))
    input_K = Input(shape=(12288,))

    # Adding Gaussian noise with 0.01 standard deviation.
    input_with_noise = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed
    x = input_with_noise
    # tensor = Flatten()(x)
    # #tensor = Reshape((1, -1))(tensor)
    # print(tensor.shape)
    # indices = Reshape((-1,2))(input_K)
    # indices = tf.dtypes.cast(indices, tf.int32)
    # print(indices.shape)
    # shape = tf.shape(tensor)
    
    # scattered_tensor = tf.gather_nd(tensor, indices)
    # x = Reshape((64,64,3))(scattered_tensor)
    # print(x.shape)

    #x = concatenate([input_with_noise, input_K])
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)

    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(tf.shape(integer_K)[0]), axis = 1)
    i1 = tf.tile(i1, [1, 12288])
    indices = tf.stack([i1,integer_K], axis=-1)
    tensor = Flatten()(x)
    shape = tf.shape(tensor)
    
    scattered_tensor = tf.gather_nd(tensor, indices)
    output_Sprime= Reshape((64,64,3))(scattered_tensor)
    #print(x.shape)
    
    if not fixed:
        return Model(inputs=[reveal_input,input_K],
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs=[reveal_input,input_K],   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    input_K = Input(shape=(12288,))
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C, input_K]) #changed
    output_Sprime = decoder([output_Cprime, input_K]) #changed

    autoencoder = Model(inputs=[input_S, input_C, input_K],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [28]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [29]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/pixel_shuffling/pixel_shuffling_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/pixel_shuffling/pixel_shuffling_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/pixel_shuffling/pixel_shuffling_reveal_model_100k_50e.hdf5')

In [30]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [31]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [32]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [33]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  input_K = np.zeros(shape=(len(input_S), 12288))
  for j in range(len(input_S)):
      input_K[j] = random.sample(range(0, 12288), 12288)

  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime, input_K])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  10.864216240500081
Encryption Time :  6.8291192281000805
Decryption Time :  5.035834916200111
pixel_s_error :  50.70576820373535
pixel_c_error:  14.052678871154786
ssim_s_error :  0.3066943556070328
ssim_c_error :  0.8923108518123627
psnr_s_error :  14.448548412322998
psnr_c_error :  25.98096694946289


In [34]:
decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

# One Block Shuffling

### Model Implementation


In [35]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = K.sum(K.square(s_true - s_pred))
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))
    input_K = Input(shape=(4096,))
  
    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(tf.shape(integer_K)[0]), axis = 1)
    i1 = tf.tile(i1, [1, 4096])
    indices = tf.stack([i1, integer_K], axis=-1)
    tensor = Reshape(( -1, 3))(input_S)
    shape = tf.shape(tensor)
    scattered_tensor = tf.scatter_nd(indices, tensor, shape)
    x = Reshape((64,64,3))(scattered_tensor)
    

    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = concatenate([input_C, x])
    
    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)
    
    return Model(inputs=[input_S, input_C, input_K], #changed
                 outputs=output_Cprime,
                 name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    
    # Reveal network
    reveal_input = Input(shape=(input_size))
    input_K = Input(shape=(4096,))

    # Adding Gaussian noise with 0.01 standard deviation.
    input_with_noise = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed
    x = input_with_noise
   
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)

    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(tf.shape(integer_K)[0]), axis = 1)
    i1 = tf.tile(i1, [1, 4096])
    indices = tf.stack([i1,integer_K], axis=-1)
    tensor = Reshape(( -1, 3))(x)
    shape = tf.shape(tensor)
    
    scattered_tensor = tf.gather_nd(tensor, indices)
    output_Sprime= Reshape((64,64,3))(scattered_tensor)
    #print(x.shape)
    
    if not fixed:
        return Model(inputs=[reveal_input,input_K],
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs=[reveal_input,input_K],   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):
    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    input_K = Input(shape=(4096,))
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C, input_K]) #changed
    output_Sprime = decoder([output_Cprime, input_K]) #changed

    autoencoder = Model(inputs=[input_S, input_C, input_K],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [36]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [37]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/one_block_shuffling/one_block_shuffling_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/one_block_shuffling/one_block_shuffling_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/one_block_shuffling/one_block_shuffling_reveal_model_100k_50e.hdf5')

In [38]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [39]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [40]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [41]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  input_K = np.zeros(shape=(len(input_S), 4096))
  for j in range(len(input_S)):
      input_K[j] = random.sample(range(0, 4096), 4096)

  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime, input_K])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  10.477397351700166
Encryption Time :  6.408893567299856
Decryption Time :  4.481946760099982
pixel_s_error :  10.393278217315673
pixel_c_error:  10.873315048217773
ssim_s_error :  0.9171652257442474
ssim_c_error :  0.859525054693222
psnr_s_error :  28.765575408935547
psnr_c_error :  27.695691299438476


In [42]:
decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

# Eight Block Shuffling

### Model Implementation

In [43]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = K.sum(K.square(s_true - s_pred))
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):

    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))
    input_K= Input(shape=(total_block,))

    batch_size = tf.shape(input_S)[0]
    # print(tf.shape(input_S))
    # block_in_one_axis =  row // block_size
    # total_block = block_in_one_axis * block_in_one_axis


    blocks = tf.image.extract_patches(input_S,sizes=[1,block_size,block_size,1],strides=[1,block_size,block_size,1],rates=[1, 1, 1, 1],padding='VALID')
    blocks = tf.reshape(blocks,[batch_size, total_block, block_size, block_size, 3])

    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(batch_size), axis = 1)
    i1 = tf.tile(i1, [1, total_block])
    indices = tf.stack([i1,integer_K], axis=-1)

    scattered = tf.scatter_nd(indices, blocks, (batch_size, total_block, block_size, block_size,3 ))

    scattered_blocks = tf.unstack(scattered, axis =1)
    scattered_tensor_row_wise = [tf.concat(scattered_blocks[i:i+block_in_one_axis], axis = -2) for i in tf.range(0,total_block,block_in_one_axis)]
    x = tf.keras.layers.concatenate(scattered_tensor_row_wise, axis = 1)


    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])

    x = concatenate([input_C, x])

    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])

    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)

    return Model(inputs=[input_S, input_C, input_K], #changed
                  outputs=output_Cprime,
                  name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    reveal_input = Input(shape=(input_size))
  
    batch_size = tf.shape(reveal_input)[0]
    # print(tf.shape(reveal_input))
    # block_in_one_axis =  row // block_size
    # total_block = block_in_one_axis * block_in_one_axis
    
    input_K= Input(shape=(total_block,))

    
    # Adding Gaussian noise with 0.01 standard deviation.
    input_with_noise = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed
    x = input_with_noise

    #x = concatenate([input_with_noise, input_K])
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)


    scattered_tensor = x
    blocks = tf.image.extract_patches(scattered_tensor,sizes=[1,block_size,block_size,1],strides=[1,block_size,block_size,1],rates=[1, 1, 1, 1],padding='VALID')
    blocks = tf.reshape(blocks,[batch_size,total_block,block_size,block_size,3])

    integer_K = tf.dtypes.cast(input_K, tf.int32)

    i1 = tf.expand_dims(tf.range(batch_size), axis = 1)
    i1 = tf.tile(i1, [1, total_block])
    indices = tf.stack([i1,integer_K], axis=-1)

    unscattered = tf.gather_nd(blocks,indices)

    unscattered_blocks = tf.unstack(unscattered, axis =1)
    unscattered_tensor_row_wise = [tf.concat(unscattered_blocks[i:i+block_in_one_axis], axis = -2) for i in range(0,total_block,block_in_one_axis)]
    output_Sprime = tf.keras.layers.concatenate(unscattered_tensor_row_wise, axis = 1)
    
    if not fixed:
        return Model(inputs=[reveal_input,input_K],
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs=[reveal_input,input_K],   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):

    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    input_K = Input(shape=(total_block,))
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C, input_K]) #changed
    output_Sprime = decoder([output_Cprime, input_K]) #changed

    autoencoder = Model(inputs=[input_S, input_C, input_K],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [44]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [45]:
# Block size to fragment the image
block_size = 8
block_in_one_axis =  64 // block_size
total_block = block_in_one_axis * block_in_one_axis

In [46]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/eight_block_shuffling/eight_block_shuffling_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/eight_block_shuffling/eight_block_shuffling_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/eight_block_shuffling/eight_block_shuffling_reveal_model_100k_50e.hdf5')

In [47]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [48]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [49]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [50]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  input_K = np.zeros(shape=(len(input_S), total_block))
  for j in range(len(input_S)):
      input_K[j] = random.sample(range(0, total_block), total_block)

  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime, input_K])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  10.57219782860002
Encryption Time :  6.16702762520008
Decryption Time :  4.30125903850003
pixel_s_error :  7.071032285690308
pixel_c_error:  8.463178348541259
ssim_s_error :  0.9454292178153991
ssim_c_error :  0.9001663506031037
psnr_s_error :  31.5910005569458
psnr_c_error :  29.845921325683594


# Key Concatenation + Eight Block Shuffling

### Model Implementation

In [59]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = K.sum(K.square(s_true - s_pred))
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):

    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))
    input_K = Input(shape=(total_block,))
    input_K_concat = Input(shape=(input_size))

    batch_size = tf.shape(input_S)[0]
   
    blocks = tf.image.extract_patches(input_S,sizes=[1,block_size,block_size,1],strides=[1,block_size,block_size,1],rates=[1, 1, 1, 1],padding='VALID')
    blocks = tf.reshape(blocks,[batch_size, total_block, block_size, block_size, 3])

    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(batch_size), axis = 1)
    i1 = tf.tile(i1, [1, total_block])
    indices = tf.stack([i1,integer_K], axis=-1)

    scattered = tf.scatter_nd(indices, blocks, (batch_size, total_block, block_size, block_size,3 ))

    scattered_blocks = tf.unstack(scattered, axis =1)
    scattered_tensor_row_wise = [tf.concat(scattered_blocks[i:i+block_in_one_axis], axis = -2) for i in tf.range(0,total_block,block_in_one_axis)]
    x = concatenate(scattered_tensor_row_wise, axis = 1)

    # Key Concatenation
    x = concatenate([x, input_K_concat])


    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = concatenate([input_C, x])
    
    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)
    
    return Model(inputs=[input_S, input_C, input_K, input_K_concat], #changed
                 outputs=output_Cprime,
                 name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    # Reveal network
    reveal_input = Input(shape=(input_size))
    input_K = Input(shape=(total_block,))
    input_K_concat = Input(shape=(input_size))

    batch_size = tf.shape(reveal_input)[0]
    # Adding Gaussian noise with 0.01 standard deviation.
    input_with_noise = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed

    x = concatenate([input_with_noise, input_K_concat])
    #x = concatenate([input_with_noise, input_K])
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)

    scattered_tensor = x
    blocks = tf.image.extract_patches(scattered_tensor,sizes=[1,block_size,block_size,1],strides=[1,block_size,block_size,1],rates=[1, 1, 1, 1],padding='VALID')
    blocks = tf.reshape(blocks,[batch_size,total_block,block_size,block_size,3])

    integer_K = tf.dtypes.cast(input_K, tf.int32)

    i1 = tf.expand_dims(tf.range(batch_size), axis = 1)
    i1 = tf.tile(i1, [1, total_block])
    indices = tf.stack([i1,integer_K], axis=-1)

    unscattered = tf.gather_nd(blocks,indices)

    unscattered_blocks = tf.unstack(unscattered, axis =1)
    unscattered_tensor_row_wise = [tf.concat(unscattered_blocks[i:i+block_in_one_axis], axis = -2) for i in range(0,total_block,block_in_one_axis)]
    output_Sprime = concatenate(unscattered_tensor_row_wise, axis = 1)

    
    if not fixed:
        return Model(inputs=[reveal_input,input_K, input_K_concat],
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs=[reveal_input,input_K, input_K_concat],   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):
    # Calculations for dividing into blocks

    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    input_K = Input(shape=(total_block,))
    input_K_concat = Input(shape=(input_size)) 
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C, input_K, input_K_concat]) #changed
    output_Sprime = decoder([output_Cprime, input_K, input_K_concat]) #changed

    autoencoder = Model(inputs=[input_S, input_C, input_K, input_K_concat],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [60]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [61]:
# Calculations for dividing into blocks
block_size = 8
block_in_one_axis =  64 // block_size
total_block = block_in_one_axis * block_in_one_axis

In [62]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/key_concatenation_eight_block_shuffling/key_concatenation_eight_block_shuffling_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/key_concatenation_eight_block_shuffling/key_concatenation_eight_block_shuffling_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/key_concatenation_eight_block_shuffling/key_concatenation_eight_block_shuffling_reveal_model_100k_50e.hdf5')

In [63]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [65]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [66]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [67]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  input_K = np.zeros(shape=(len(input_S), total_block))
  for j in range(len(input_S)):
      input_K[j] = random.sample(range(0, total_block), total_block)

  input_K_concat = np.random.randint(256, size=(len(input_S), 64, 64, 3))
  input_K_concat = input_K_concat/255.

  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C, input_K, input_K_concat])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C, input_K, input_K_concat])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime, input_K, input_K_concat])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  10.825053038400165
Encryption Time :  6.400785297500079
Decryption Time :  4.4795099464999115
pixel_s_error :  8.326627159118653
pixel_c_error:  11.343319034576416
ssim_s_error :  0.9453914999961853
ssim_c_error :  0.8998486161231994
psnr_s_error :  30.00141296386719
psnr_c_error :  27.895423889160156


In [ ]:
decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

# Four Block Shuffling

### Model Implementation

In [70]:
# Variable used to weight the losses of the secret and cover images (See paper for more details)
beta = 1.0
    
# Loss for reveal network
def rev_loss(y_true, y_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(y_true - y_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[:,:,:,0:3], y_true[:,:,:,3:6]
    s_pred, c_pred = y_pred[:,:,:,0:3], y_pred[:,:,:,3:6]
    s_loss = K.sum(K.square(s_true - s_pred))
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss


# Returns the encoder as a Keras model, composed by Preparation and Hiding Networks.
def make_encoder(input_size):

    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size))
    input_K= Input(shape=(total_block,))

    batch_size = tf.shape(input_S)[0]
    # print(tf.shape(input_S))
    # block_in_one_axis =  row // block_size
    # total_block = block_in_one_axis * block_in_one_axis


    blocks = tf.image.extract_patches(input_S,sizes=[1,block_size,block_size,1],strides=[1,block_size,block_size,1],rates=[1, 1, 1, 1],padding='VALID')
    blocks = tf.reshape(blocks,[batch_size, total_block, block_size, block_size, 3])

    integer_K = tf.dtypes.cast(input_K, tf.int32)
    i1 = tf.expand_dims(tf.range(batch_size), axis = 1)
    i1 = tf.tile(i1, [1, total_block])
    indices = tf.stack([i1,integer_K], axis=-1)

    scattered = tf.scatter_nd(indices, blocks, (batch_size, total_block, block_size, block_size,3 ))

    scattered_blocks = tf.unstack(scattered, axis =1)
    scattered_tensor_row_wise = [tf.concat(scattered_blocks[i:i+block_in_one_axis], axis = -2) for i in tf.range(0,total_block,block_in_one_axis)]
    x = tf.keras.layers.concatenate(scattered_tensor_row_wise, axis = 1)


    # Preparation Network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_4x4')(x) #changed
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep0_5x5')(x) #changed
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_prep1_5x5')(x)
    x = concatenate([x3, x4, x5])

    x = concatenate([input_C, x])

    # Hiding network
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid0_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid1_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid2_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid3_5x5')(x)
    x = concatenate([x3, x4, x5])

    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_hid4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_hid5_5x5')(x)
    x = concatenate([x3, x4, x5])

    output_Cprime = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_C')(x)

    return Model(inputs=[input_S, input_C, input_K], #changed
                  outputs=output_Cprime,
                  name = 'Encoder')

# Returns the decoder as a Keras model, composed by the Reveal Network
def make_decoder(input_size, fixed=False):
    reveal_input = Input(shape=(input_size))
  
    batch_size = tf.shape(reveal_input)[0]
    # print(tf.shape(reveal_input))
    # block_in_one_axis =  row // block_size
    # total_block = block_in_one_axis * block_in_one_axis
    
    input_K= Input(shape=(total_block,))

    
    # Adding Gaussian noise with 0.01 standard deviation.
    input_with_noise = GaussianNoise(0.01, name='output_C_noise')(reveal_input) #changed
    x = input_with_noise

    #x = concatenate([input_with_noise, input_K])
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_3x3')(x) #changed
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev0_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev1_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev2_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev3_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x3 = Conv2D(50, (3, 3), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_3x3')(x)
    x4 = Conv2D(10, (4, 4), strides = (1, 1), padding='same', activation='relu', name='conv_rev4_4x4')(x)
    x5 = Conv2D(5, (5, 5), strides = (1, 1), padding='same', activation='relu', name='conv_rev5_5x5')(x)
    x = concatenate([x3, x4, x5])
    
    x = Conv2D(3, (3, 3), strides = (1, 1), padding='same', activation='relu', name='output_S')(x)


    scattered_tensor = x
    blocks = tf.image.extract_patches(scattered_tensor,sizes=[1,block_size,block_size,1],strides=[1,block_size,block_size,1],rates=[1, 1, 1, 1],padding='VALID')
    blocks = tf.reshape(blocks,[batch_size,total_block,block_size,block_size,3])

    integer_K = tf.dtypes.cast(input_K, tf.int32)

    i1 = tf.expand_dims(tf.range(batch_size), axis = 1)
    i1 = tf.tile(i1, [1, total_block])
    indices = tf.stack([i1,integer_K], axis=-1)

    unscattered = tf.gather_nd(blocks,indices)

    unscattered_blocks = tf.unstack(unscattered, axis =1)
    unscattered_tensor_row_wise = [tf.concat(unscattered_blocks[i:i+block_in_one_axis], axis = -2) for i in range(0,total_block,block_in_one_axis)]
    output_Sprime = tf.keras.layers.concatenate(unscattered_tensor_row_wise, axis = 1)
    
    if not fixed:
        return Model(inputs=[reveal_input,input_K],
                     outputs=output_Sprime,
                     name = 'Decoder')
    # else:
    #     return Container(inputs=reveal_input,
    #                      outputs=output_Sprime,
    #                      name = 'DecoderFixed')                    # Changed
    else:
      return Network(inputs=[reveal_input,input_K],   #changed
                      outputs=output_Sprime,
                      name = 'DecoderFixed')

# Full model.
def make_model(input_size):

    input_S = Input(shape=(input_size))
    input_C= Input(shape=(input_size)) 
    input_K = Input(shape=(total_block,))
    
    encoder = make_encoder(input_size)
    
    decoder = make_decoder(input_size)
    decoder.compile(optimizer='adam', loss=rev_loss)
    decoder.trainable = False
    
    output_Cprime = encoder([input_S, input_C, input_K]) #changed
    output_Sprime = decoder([output_Cprime, input_K]) #changed

    autoencoder = Model(inputs=[input_S, input_C, input_K],
                        outputs=concatenate([output_Sprime, output_Cprime]))
    autoencoder.compile(optimizer='adam', loss=full_loss)
    return encoder, decoder, autoencoder

## Test 

In [69]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(123)
random.seed(121)
tf.random.set_seed(85)

In [71]:
# Block size to fragment the image
block_size = 4
block_in_one_axis =  64 // block_size
total_block = block_in_one_axis * block_in_one_axis

In [72]:
encoder_model, reveal_model, autoencoder_model = make_model(input_S.shape[1:])
autoencoder_model.load_weights('/content/drive/My Drive/Final_Model/four_block_shuffling/four_block_shuffling_autoencoder_model_100k_50e.hdf5')
encoder_model.load_weights('/content/drive/My Drive/Final_Model/four_block_shuffling/four_block_shuffling_encoder_model_100k_50e.hdf5')
reveal_model.load_weights('/content/drive/My Drive/Final_Model/four_block_shuffling/four_block_shuffling_reveal_model_100k_50e.hdf5')

In [73]:
def ssim(input_S, input_C, decoded_S, decoded_C):
  ssim_S = tf.reduce_mean(tf.image.ssim(input_S, decoded_S, 1.0))
  ssim_C = tf.reduce_mean(tf.image.ssim(input_C, decoded_C, 1.0))
  
  return ssim_S.numpy(), ssim_C.numpy()

In [74]:
def psnr(input_S, input_C, decoded_S, decoded_C):
  psnr_S = tf.reduce_mean(tf.image.psnr(input_S, decoded_S, max_val=1.0))
  psnr_C = tf.reduce_mean(tf.image.psnr(input_C, decoded_C, max_val=1.0))

  return psnr_S.numpy(), psnr_C.numpy()

In [75]:
def pixel_errors(input_S, input_C, decoded_S, decoded_C):
    """Calculates mean of Sum of Squared Errors per pixel for cover and secret images. """
    rmse_Spixel = np.sqrt(np.mean(np.square(255*(input_S - decoded_S))))
    rmse_Cpixel = np.sqrt(np.mean(np.square(255*(input_C - decoded_C))))
    
    return rmse_Spixel, rmse_Cpixel

### Total prediction time

In [76]:
total_prediction_time = 0
encryption_time = 0
decryption_time = 0
pixel_s_error = 0
pixel_c_error = 0
ssim_s_error = 0
ssim_c_error = 0
psnr_s_error = 0
psnr_c_error = 0

for i in range(loop_no):
  input_K = np.zeros(shape=(len(input_S), total_block))
  for j in range(len(input_S)):
      input_K[j] = random.sample(range(0, total_block), total_block)

  start = time.perf_counter()
  decoded = autoencoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  total_prediction_time = total_prediction_time + (end-start)

  decoded_S, decoded_C = decoded[...,0:3], decoded[...,3:6]

  start = time.perf_counter()
  C_prime = encoder_model.predict([input_S, input_C, input_K])
  end = time.perf_counter()
  encryption_time = encryption_time + (end-start)

  start = time.perf_counter()
  S_prime = reveal_model.predict([C_prime, input_K])
  end = time.perf_counter()
  decryption_time = decryption_time + (end-start)

  s_error, c_error = pixel_errors(input_S, input_C, decoded_S, decoded_C)
  pixel_s_error = pixel_s_error +  s_error
  pixel_c_error = pixel_c_error +  c_error

  s_error, c_error = ssim(input_S, input_C, decoded_S, decoded_C)
  ssim_s_error = ssim_s_error +  s_error
  ssim_c_error = ssim_c_error +  c_error

  s_error, c_error = psnr(input_S, input_C, decoded_S, decoded_C)
  psnr_s_error = psnr_s_error +  s_error
  psnr_c_error = psnr_c_error +  c_error
 
print("Total Prediction Time : ", total_prediction_time/loop_no) 
print("Encryption Time : ", encryption_time/loop_no) 
print("Decryption Time : ", decryption_time/loop_no)
print("pixel_s_error : ", pixel_s_error/loop_no)  
print("pixel_c_error: ", pixel_c_error/loop_no) 
print("ssim_s_error : ", ssim_s_error/loop_no) 
print("ssim_c_error : ", ssim_c_error/loop_no) 
print("psnr_s_error : ", psnr_s_error/loop_no) 
print("psnr_c_error : ", psnr_c_error/loop_no) 

Total Prediction Time :  10.795643633500367
Encryption Time :  6.291232754400153
Decryption Time :  4.400865177700507
pixel_s_error :  5.7190908908844
pixel_c_error:  8.893681621551513
ssim_s_error :  0.9605802357196808
ssim_c_error :  0.8915749728679657
psnr_s_error :  33.516607666015624
psnr_c_error :  29.37933521270752
